In [1]:
import os
import openai
from openai import OpenAI
import time


class OllamaManager:
    def __init__(self, model="llama3.1:latest", base_url="http://localhost:11434/v1"):
        """
        Initializes the OllamaManager:
        - Installs Ollama
        - Starts Ollama server
        - Pulls the default Llama model
        - Sets up environment variables
        """
        self.model = model
        self.base_url = base_url
        self.initialized = False

        self.setup()

    def setup(self):
        """Sets up Ollama, pulls the model, and lists installed models."""
        if not self.initialized:
            self.install_ollama()
            self.start_ollama_server()
            self.pull_model(self.model)
            self.list_installed_models()
            self.agent = llmAgent(base_url=self.base_url, model=self.model, api_key="none")
            self.initialized = True
        else:
            print("🟢 Setpu has already been initialized.")

    def install_ollama(self):
        """Installs Ollama on the system."""
        print("📌 Installing Ollama...")
        os.system("curl -fsSL https://ollama.com/install.sh | sh")
        os.system("pip install openai pandas requests")
        print("✅ Installation completed.")

    def start_ollama_server(self):
        """Starts the Ollama servre."""
        print("📌 Setting up environment variables...")
        os.environ["OLLAMA_HOST"] = "0.0.0.0"
        os.environ["no_proxy"] = "localhost,127.0.0.1"

        print("📌 Starting Ollama server in the background...")
        os.system("nohup ollama serve &")
        print("🕒 Waiting for the server to start...")
        time.sleep(5)  # wait some time for the server to start

    def pull_model(self, model_name):
        """Pulls the specified Llama model."""
        print(f"📌 Pulling model: {model_name}...")
        os.system(f"ollama pull {model_name}")
        print(f"✅ Model {model_name} pulled successfully.")

    def list_installed_models(self):
        """Lists all installed models in Ollama."""
        print("📌 Listing installed models...")
        !ollama list

    def change_model(self, new_model):
        """Changes the model without re-running setup."""
        print(f"🔄 Changing model to: {new_model}...")
        self.model = new_model
        self.pull_model(new_model)
        self.agent = llmAgent(base_url=self.base_url, model=self.model, api_key="none")
        print(f"✅ Model switched to: {new_model}")


class llmAgent:
    def __init__(self, base_url, api_key, model, temperature=0):
        """
        Initializes the LLM Agent for interacting with the local model.
        """
        self.client = OpenAI(
            base_url=base_url,
            api_key=api_key
        )
        self.model = model
        self.temperature = temperature

    def ask_question(self, question, system_role_content, temperature=None):
        """
        Asks a question to the model and returns the response.
        """
        if temperature is None:
            temperature = self.temperature

        response = self.client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": system_role_content},
                {"role": "user", "content": question},
            ],
            temperature=temperature,
        )
        return response.choices[0].message.content


# -------------------- Example Usage --------------------

if __name__ == "__main__":
    # Initialize OllamaManager
    ollama = OllamaManager(model="llama3.1:latest")


📌 Installing Ollama...
✅ Installation completed.
📌 Setting up environment variables...
📌 Starting Ollama server in the background...
🕒 Waiting for the server to start...
📌 Pulling model: llama3.1:latest...
✅ Model llama3.1:latest pulled successfully.
📌 Listing installed models...
NAME               ID              SIZE      MODIFIED               
llama3.1:latest    46e0c10c039e    4.9 GB    Less than a second ago    


In [2]:
ollama.list_installed_models()

📌 Listing installed models...
NAME               ID              SIZE      MODIFIED      
llama3.1:latest    46e0c10c039e    4.9 GB    7 seconds ago    


In [3]:
# role = "You are a highly skilled mathematics assistant, capable of solving problems, explaining concepts, and providing step-by-step solutions in various mathematical domains, including algebra, calculus, geometry, statistics, and more."
# role = "You are a mathematics assistant specializing in solving mathematical problems across different domains. You provide step-by-step explanations, visual breakdowns when applicable, and concise yet thorough solutions. Ensure clarity and correctness in your responses."
# role = "You are a mathematics assistant trained to solve complex mathematical problems, assist in theorem proving, and provide in-depth explanations across topics such as real analysis, number theory, linear algebra, and differential equations. Present rigorous justifications for solutions and use LaTeX formatting where necessary."
# role = "You are an expert mathematics tutor, assisting students by explaining mathematical concepts clearly and intuitively. Adapt your explanations based on the learner's level, provide real-world examples where possible, and guide students through problem-solving with a step-by-step approach."
role = "You are a logical reasoning and mathematics assistant. Your goal is to solve word problems with a step-by-step approach, ensuring clarity, correctness, and logical consistency."

query = "If I hang 5 shirts outside and it takes them 5 hours to dry, how long would it take to dry 30 shirts?"


ollama.change_model("llama3.1:latest")
answer = ollama.agent.ask_question(query, role)
print("\n🔹 Response from llama3.1:")
print(answer)


ollama.change_model("mistral:7b")
answer = ollama.agent.ask_question(query, role)
print("\n🔹 Response from mistral:7b:")
print(answer)


ollama.change_model("llama3.2:latest")
answer = ollama.agent.ask_question(query, role)
print("\n🔹 Response from llama3.2:latest:")
print(answer)

🔄 Changing model to: llama3.1:latest...
📌 Pulling model: llama3.1:latest...
✅ Model llama3.1:latest pulled successfully.
✅ Model switched to: llama3.1:latest

🔹 Response from llama3.1:
Let's break down the problem step by step:

1. We know that 5 shirts take 5 hours to dry.
2. To find out how long it takes for one shirt to dry, we can divide the total time (5 hours) by the number of shirts (5). This gives us:
   Time per shirt = Total time / Number of shirts
                    = 5 hours / 5 shirts
                    = 1 hour per shirt

3. Now that we know it takes 1 hour for one shirt to dry, we can find out how long it would take to dry 30 shirts by multiplying the number of shirts (30) by the time per shirt (1 hour):
   Time to dry 30 shirts = Number of shirts × Time per shirt
                           = 30 shirts × 1 hour/shirt
                           = 30 hours

Therefore, it would take 30 hours for 30 shirts to dry.
🔄 Changing model to: mistral:7b...
📌 Pulling model: mistral

In [4]:
ollama.list_installed_models()

📌 Listing installed models...
NAME               ID              SIZE      MODIFIED       
llama3.2:latest    a80c4f17acd5    2.0 GB    16 seconds ago    
mistral:7b         f974a74358d6    4.1 GB    2 minutes ago     
llama3.1:latest    46e0c10c039e    4.9 GB    4 minutes ago     
